importacion de librerias


In [89]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [90]:
df = pd.read_csv('./data/2024_Accidentalidad.csv', delimiter=';')


In [91]:
df.sample(10)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
6809,2024S006401,20/02/2024,19:15:00,PLAZA ELIPTICA,2,12,USERA,Colisión frontal,Despejado,Turismo,Conductor,De 30 a 34 años,Hombre,NaN,NaN,439135.0,4470781.0,N,NaN
8386,2024S008084,04/03/2024,18:18:00,"CMNO. SUERTE, 4",4,18,VILLA DE VALLECAS,Atropello a persona,Despejado,Turismo,Conductor,De 50 a 54 años,Hombre,14.0,Sin asistencia sanitaria,447352.0,4469705.0,N,NaN
12716,2024S012597,06/04/2024,19:00:00,CALL. ZURBANO / CALL. JOSE ABASCAL,88,7,CHAMBERÍ,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 55 a 59 años,Mujer,NaN,NaN,441234.0,4476614.0,N,NaN
39584,2024S029966,28/10/2024,14:10:00,"AVDA. LOS ROSALES, 7",7,12,USERA,Choque contra obstáculo fijo,Despejado,Furgoneta,Conductor,De 55 a 59 años,Hombre,NaN,NaN,441156.0,4468881.0,N,NaN
21611,2024S019119,07/06/2024,17:30:00,"CALL. LOMA, 2",2,7,CHAMBERÍ,Alcance,Despejado,Turismo,Conductor,De 40 a 44 años,Hombre,14.0,Sin asistencia sanitaria,439308.0,4477400.0,N,NaN
27042,2024S022305,17/07/2024,15:30:00,CALL. FEDERICO CARLOS SAINZ DE ROBLES / CALL. ...,3,9,MONCLOA-ARAVACA,Colisión lateral,Despejado,Motocicleta hasta 125cc,Conductor,De 40 a 44 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,438885.0,4479564.0,N,NaN
38447,2024S029249,19/10/2024,20:51:00,"AUTOV. A-2, +00500S",+00500S,15,CIUDAD LINEAL,Alcance,Despejado,Turismo,Pasajero,De 18 a 20 años,Mujer,14.0,Sin asistencia sanitaria,444594.0,4477610.0,N,NaN
7689,2024S007331,28/02/2024,16:25:00,AUTOV. M-11 / AUTOV. M-30,0,15,CIUDAD LINEAL,Colisión múltiple,Despejado,Turismo,Conductor,De 35 a 39 años,Mujer,14.0,Sin asistencia sanitaria,443318.0,4481720.0,N,NaN
33455,2024S026259,16/09/2024,9:19:00,"CALL. MELCHOR FERNANDEZ ALMAGRO, 103",103,8,FUENCARRAL-EL PARDO,Colisión fronto-lateral,Despejado,Motocicleta hasta 125cc,Conductor,De 30 a 34 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,440324.0,4480915.0,N,NaN
38129,2024S029021,17/10/2024,19:25:00,"PASEO. ESPERANZA, 49",49,2,ARGANZUELA,Colisión lateral,Despejado,Ciclomotor,Conductor,De 30 a 34 años,Hombre,6.0,Asistencia sanitaria inmediata en centro de sa...,440377.0,4472242.0,N,NaN


Analisis del dataset


https://datos.madrid.es/FWProjects/egob/Catalogo/Seguridad/Ficheros/Estructura_ConjuntoDatos_Accidentesv2.pdf


Lo primero que nos damos cuenta cuando leemos el anterior documento es que el año del accidente no viene en la fecha, si no que viene en los 4 primeros char del numero del expediente.


**localizacion** y **numero**: con la localizacion y el numero (numero de la calle) nos parece interesante quedarnosla, pero solo para la parte de entendimiento de los datos del cluster, ya que para el claster es mas util las coordenadas utm


para el codigo del distrito nos va a servir para hacer el cluster, pero como el numero en si no nos dice nada, pues entonces lo que vamos a hacer un mapeo del distrito para entender y tener la correlacion


para el tipo de accidente y para el estado metereologico haremos un mapeo simple de las 13 posibles variables (mapping)


en tipo de vehiculo habia demasiados valroes unicos (33) por lo que hemos intentado generalizar un poco para poder tratar con esta columna, haciendo un mapeo y un posible posterior onehotencoding


para el **rango de edad** vamos


Entendimiento del Dataset


Limpieza


Explicacion de las acciones realizadas


In [92]:
len(df['tipo_vehiculo'].unique())


33

In [93]:
df['tipo_vehiculo'].value_counts()

tipo_vehiculo
Turismo                              27406
Motocicleta hasta 125cc               2668
Furgoneta                             2667
Motocicleta > 125cc                   1469
Autobús                               1264
Camión rígido                          973
Todo terreno                           706
Bicicleta                              670
VMU eléctrico                          665
Ciclomotor                             387
Bicicleta EPAC (pedaleo asistido)      211
Tractocamión                           187
Maquinaria de obras                    151
Otros vehículos con motor              131
Vehículo articulado                     70
Autobús articulado                      69
Sin especificar                         40
Cuadriciclo ligero                      32
Autobus EMT                             17
Ciclo                                   13
Cuadriciclo no ligero                   12
Autocaravana                            11
Otros vehículos sin motor               

In [94]:
map_vehiculos = {
    'Turismo': 'Turismos y vehículos ligeros',
    'Todo terreno': 'Turismos y vehículos ligeros',
    'Autocaravana': 'Turismos y vehículos ligeros',
    'Motocicleta hasta 125cc': 'Motocicletas y ciclomotores',
    'Motocicleta > 125cc': 'Motocicletas y ciclomotores',
    'Ciclomotor': 'Motocicletas y ciclomotores',
    'Bicicleta EPAC (pedaleo asistido)': 'Motocicletas y ciclomotores',
    'Ciclomotor de tres ruedas': 'Motocicletas y ciclomotores',
    'Moto de tres ruedas hasta 125cc': 'Motocicletas y ciclomotores',
    'Moto de tres ruedas > 125cc': 'Motocicletas y ciclomotores',
    'Ciclo de motor L1e-A': 'Motocicletas y ciclomotores',
    'Ciclomotor de dos ruedas L1e-B': 'Motocicletas y ciclomotores',
    'Furgoneta': 'Vehículos de carga',
    'Camión rígido': 'Vehículos de carga',
    'Tractocamión': 'Vehículos de carga',
    'Vehículo articulado': 'Vehículos de carga',
    'Semiremolque': 'Vehículos de carga',
    'Remolque': 'Vehículos de carga',
    'Autobús': 'Vehículos de transporte público',
    'Autobús articulado': 'Vehículos de transporte público',
    'Autobus EMT': 'Vehículos de transporte público',
    'Bicicleta': 'Bicicletas y ciclos no motorizados',
    'Ciclo': 'Bicicletas y ciclos no motorizados',
    'Patinete no eléctrico': 'Bicicletas y ciclos no motorizados',
    'VMU eléctrico': 'Vehículos eléctricos ligeros',
    'Maquinaria de obras': 'Vehículos especializados',
    'Ambulancia SAMUR': 'Vehículos especializados',
    'Otros vehículos con motor': 'Vehículos especializados',
    'Otros vehículos sin motor': 'Vehículos especializados',
    'Sin especificar': 'Sin clasificar/especificar',
    'Cuadriciclo ligero': 'Sin clasificar/especificar',
    'Cuadriciclo no ligero': 'Sin clasificar/especificar'
}

In [95]:
df['tipo_vehiculo'] = df['tipo_vehiculo'].map(map_vehiculos)

Mapeo de rango de edades


In [ ]:
map_edad = {
    'Desconocido': -1,
    'Menor de 5 años': 0,
    'De 6 a 9 años': 0,
    'De 10 a 14 años': 10,
    'De 15 a 17 años': 10,
    'De 18 a 20 años': 18,
    'De 21 a 24 años': 18,
    'De 25 a 29 años': 25,
    'De 30 a 34 años': 25,
    'De 35 a 39 años': 25,
    'De 40 a 44 años': 40,
    'De 45 a 49 años': 40,
    'De 50 a 54 años': 40,
    'De 55 a 59 años': 40,
    'De 60 a 64 años': 60,
    'De 65 a 69 años': 60,
    'De 70 a 74 años': 60,
    'Más de 74 años': 60
}

In [97]:
df['rango_edad'] = df['rango_edad'].map(map_edad)

Mapeo de estado meteorológico


In [98]:
df = df.fillna({
    'estado_meteorológico': 'Se desconoce'
})

print(df['estado_meteorológico'].unique())

map_est_met = {
    'Se Desconoce': -1,
    'Despejado': 0,
    'Nublado': 1,
    'Lluvia débil': 2,
    'LLuvia intensa': 3,
    'Granizando': 4,
    'Nevando': 5
}

['Lluvia débil' 'Despejado' 'Se desconoce' 'Nublado' 'LLuvia intensa'
 'Granizando' 'Nevando']


Mapeo Lesividad


In [99]:
print(df['lesividad'].unique())

['Ingreso inferior o igual a 24 horas' 'Sin asistencia sanitaria'
 'Asistencia sanitaria sólo en el lugar del accidente' nan
 'Atención en urgencias sin posterior ingreso'
 'Ingreso superior a 24 horas'
 'Asistencia sanitaria inmediata en centro de salud o mutua'
 'Asistencia sanitaria ambulatoria con posterioridad' 'Fallecido 24 horas']


In [100]:
df = df.fillna({
    'lesividad': 'Sin asistencia sanitaria',
    'cod_lesividad': 0
})

map_lesividad = {
    'Sin asistencia sanitaria': 'NONE',
    'Atención en urgencias sin posterior ingreso': 'LEVE',
    'Ingreso inferior o igual a 24 horas': 'LEVE',
    'Asistencia sanitaria ambulatoria con posterioridad': 'LEVE',
    'Asistencia sanitaria inmediata en centro de salud o mutua': 'LEVE',
    'Asistencia sanitaria sólo en el lugar del accidente': 'LEVE',
    'Ingreso inferior o igual a 24 horas': 'GRAVE',
    'Fallecido 24 horas': 'FALLECIDO'
}

In [101]:
df['lesividad'] = df['lesividad'].map(map_lesividad)

In [116]:
df['lesividad'].value_counts()

lesividad
NONE         30938
LEVE          7234
GRAVE         1494
FALLECIDO       24
Name: count, dtype: int64

Mapeo Alcohol y drogas


In [102]:
df = df.fillna({
    'positiva_droga': 0
})

map_alcohol = {
    'N': 0,
    'S': 1
}

df['positiva_alcohol'] = df['positiva_alcohol'].map(map_alcohol)

In [103]:
print('valores de positivos en alcohol: \n {} \n valores de positivos en droga: \n {}'.format(
    df['positiva_alcohol'].value_counts(),df['positiva_droga'].value_counts()
    ))

valores de positivos en alcohol: 
 positiva_alcohol
0.0    38878
1.0     1149
Name: count, dtype: int64 
 valores de positivos en droga: 
 positiva_droga
0.0    39995
1.0      170
Name: count, dtype: int64


In [104]:
df.head(10)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,Lluvia débil,Motocicletas y ciclomotores,Conductor,55,Hombre,2.0,GRAVE,444913.0,4481427.0,0.0,0.0
1,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,Lluvia débil,Turismos y vehículos ligeros,Conductor,55,Mujer,14.0,NONE,444913.0,4481427.0,0.0,0.0
2,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,Lluvia débil,Bicicletas y ciclos no motorizados,Conductor,25,Hombre,7.0,LEVE,440123.0,4475170.0,0.0,0.0
3,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,Lluvia débil,Motocicletas y ciclomotores,Conductor,35,Hombre,14.0,NONE,440123.0,4475170.0,0.0,0.0
4,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,Despejado,Turismos y vehículos ligeros,Conductor,25,Hombre,0.0,NONE,440137.0,4475721.0,0.0,0.0
5,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,Despejado,Turismos y vehículos ligeros,Conductor,55,Hombre,0.0,NONE,440137.0,4475721.0,0.0,0.0
6,2023S040312,25/02/2024,17:20:00,"CALL. PINOS ALTA, 93",93,6,TETUÁN,Colisión lateral,Despejado,Vehículos de carga,Conductor,25,Hombre,14.0,NONE,440709.0,4480102.0,0.0,0.0
7,2023S040312,25/02/2024,17:20:00,"CALL. PINOS ALTA, 93",93,6,TETUÁN,Colisión lateral,Despejado,Vehículos especializados,Conductor,50,Hombre,14.0,NONE,440709.0,4480102.0,0.0,0.0
8,2023S040316,07/03/2024,14:25:00,CTRA. BARRIO DE LA FORTUNA / CALL. PINAR DE SA...,14,10,LATINA,Alcance,Lluvia débil,Vehículos de carga,Conductor,60,Hombre,0.0,NONE,433983.0,4468473.0,0.0,0.0
9,2023S040316,07/03/2024,14:25:00,CTRA. BARRIO DE LA FORTUNA / CALL. PINAR DE SA...,14,10,LATINA,Alcance,Lluvia débil,Turismos y vehículos ligeros,Conductor,25,Hombre,0.0,NONE,433983.0,4468473.0,0.0,0.0


In [105]:
df['rango_edad'].value_counts()

rango_edad
-1     4316
 45    4107
 30    3919
 25    3885
 40    3845
 35    3796
 50    3698
 55    2869
 21    2627
 60    1990
 18    1268
 74     963
 65     956
 70     576
 15     374
 0      367
 10     349
 6      260
Name: count, dtype: int64

In [106]:
df.columns

Index(['num_expediente', 'fecha', 'hora', 'localizacion', 'numero',
       'cod_distrito', 'distrito', 'tipo_accidente', 'estado_meteorológico',
       'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'sexo', 'cod_lesividad',
       'lesividad', 'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol',
       'positiva_droga'],
      dtype='object')

In [107]:
df['cod_lesividad'].unique()

array([ 2., 14.,  7.,  0.,  1.,  3.,  6.,  5.,  4.])

In [108]:
df['cod_lesividad'] = df['cod_lesividad'].astype(int)

In [109]:
df['year'] = df['num_expediente'].astype(str).str[:4].astype(int)
df['id'] = df['num_expediente'].astype(str).str[-6:]

In [110]:
df['year'].value_counts()

year
2024    40154
2023       11
Name: count, dtype: int64

In [111]:
df = pd.get_dummies(df, columns=['tipo_persona'])

In [112]:
df.head(10)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,...,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,year,id,tipo_persona_Conductor,tipo_persona_Pasajero,tipo_persona_Peatón
0,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,Lluvia débil,Motocicletas y ciclomotores,...,GRAVE,444913.0,4481427.0,0.0,0.0,2023,040280,True,False,False
1,2023S040280,04/01/2024,14:09:00,AVDA. NICETO ALCALA ZAMORA / AUTOV. M-11,3,16,HORTALEZA,Colisión fronto-lateral,Lluvia débil,Turismos y vehículos ligeros,...,NONE,444913.0,4481427.0,0.0,0.0,2023,040280,True,False,False
2,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,Lluvia débil,Bicicletas y ciclos no motorizados,...,LEVE,440123.0,4475170.0,0.0,0.0,2023,040309,True,False,False
3,2023S040309,15/02/2024,14:05:00,CALL. TESORO / CALL. MINAS,18,1,CENTRO,Colisión fronto-lateral,Lluvia débil,Motocicletas y ciclomotores,...,NONE,440123.0,4475170.0,0.0,0.0,2023,040309,True,False,False
4,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,Despejado,Turismos y vehículos ligeros,...,NONE,440137.0,4475721.0,0.0,0.0,2023,040310,True,False,False
5,2023S040310,18/02/2024,10:40:00,GTA. RUIZ JIMENEZ / CALL. SAN BERNARDO,3,7,CHAMBERÍ,Colisión lateral,Despejado,Turismos y vehículos ligeros,...,NONE,440137.0,4475721.0,0.0,0.0,2023,040310,True,False,False
6,2023S040312,25/02/2024,17:20:00,"CALL. PINOS ALTA, 93",93,6,TETUÁN,Colisión lateral,Despejado,Vehículos de carga,...,NONE,440709.0,4480102.0,0.0,0.0,2023,040312,True,False,False
7,2023S040312,25/02/2024,17:20:00,"CALL. PINOS ALTA, 93",93,6,TETUÁN,Colisión lateral,Despejado,Vehículos especializados,...,NONE,440709.0,4480102.0,0.0,0.0,2023,040312,True,False,False
8,2023S040316,07/03/2024,14:25:00,CTRA. BARRIO DE LA FORTUNA / CALL. PINAR DE SA...,14,10,LATINA,Alcance,Lluvia débil,Vehículos de carga,...,NONE,433983.0,4468473.0,0.0,0.0,2023,040316,True,False,False
9,2023S040316,07/03/2024,14:25:00,CTRA. BARRIO DE LA FORTUNA / CALL. PINAR DE SA...,14,10,LATINA,Alcance,Lluvia débil,Turismos y vehículos ligeros,...,NONE,433983.0,4468473.0,0.0,0.0,2023,040316,True,False,False


In [113]:
df.columns

Index(['num_expediente', 'fecha', 'hora', 'localizacion', 'numero',
       'cod_distrito', 'distrito', 'tipo_accidente', 'estado_meteorológico',
       'tipo_vehiculo', 'rango_edad', 'sexo', 'cod_lesividad', 'lesividad',
       'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol',
       'positiva_droga', 'year', 'id', 'tipo_persona_Conductor',
       'tipo_persona_Pasajero', 'tipo_persona_Peatón'],
      dtype='object')

In [ ]:
mapeo_accidentes = {
    'NONE':0,
    'LEVE':1,
    'GRAVE':2,
    'FALLECIDO':3
}

df['lesividad'] = df['lesividad'].map(mapeo_accidentes)

In [ ]:
map_tipo_accidente = {
    'Colisión fronto-lateral': 'Colision doble',
    'Alcance': 'Alcance',
    'Colisión lateral': 'Colision doble',
    'Choque contra obstáculo fijo':'Choque',
    'Atropello a persona': 'Atropello a persona',
    'Caída':'Caida',
    'Colisión múltiple':'Colision multiple',
    'Colisión frontal':'Colision doble',
    'Otro':'Otras Causas',
    'Solo salida de la vía': 'Otras Causas',
    'Vuelco': 'Vuelco',
    'Atropello a animal': 'Otras Causas'
}

df['tipo_accidente'] = df['tipo_accidente'].map(map_tipo_accidente)

In [114]:
df_entrenamiento = df.drop(columns=['num_expediente', 'localizacion', 'numero', 'distrito', 'lesividad'], axis=1)

In [115]:
df_entrenamiento.head(10)

,fecha,hora,cod_distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,rango_edad,sexo,cod_lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,year,id,tipo_persona_Conductor,tipo_persona_Pasajero,tipo_persona_Peatón
0,04/01/2024,14:09:00,16,Colisión fronto-lateral,Lluvia débil,Motocicletas y ciclomotores,55,Hombre,2,444913.0,4481427.0,0.0,0.0,2023,040280,True,False,False
1,04/01/2024,14:09:00,16,Colisión fronto-lateral,Lluvia débil,Turismos y vehículos ligeros,55,Mujer,14,444913.0,4481427.0,0.0,0.0,2023,040280,True,False,False
2,15/02/2024,14:05:00,1,Colisión fronto-lateral,Lluvia débil,Bicicletas y ciclos no motorizados,25,Hombre,7,440123.0,4475170.0,0.0,0.0,2023,040309,True,False,False
3,15/02/2024,14:05:00,1,Colisión fronto-lateral,Lluvia débil,Motocicletas y ciclomotores,35,Hombre,14,440123.0,4475170.0,0.0,0.0,2023,040309,True,False,False
4,18/02/2024,10:40:00,7,Colisión lateral,Despejado,Turismos y vehículos ligeros,25,Hombre,0,440137.0,4475721.0,0.0,0.0,2023,040310,True,False,False
5,18/02/2024,10:40:00,7,Colisión lateral,Despejado,Turismos y vehículos ligeros,55,Hombre,0,440137.0,4475721.0,0.0,0.0,2023,040310,True,False,False
6,25/02/2024,17:20:00,6,Colisión lateral,Despejado,Vehículos de carga,25,Hombre,14,440709.0,4480102.0,0.0,0.0,2023,040312,True,False,False
7,25/02/2024,17:20:00,6,Colisión lateral,Despejado,Vehículos especializados,50,Hombre,14,440709.0,4480102.0,0.0,0.0,2023,040312,True,False,False
8,07/03/2024,14:25:00,10,Alcance,Lluvia débil,Vehículos de carga,60,Hombre,0,433983.0,4468473.0,0.0,0.0,2023,040316,True,False,False
9,07/03/2024,14:25:00,10,Alcance,Lluvia débil,Turismos y vehículos ligeros,25,Hombre,0,433983.0,4468473.0,0.0,0.0,2023,040316,True,False,False
